# Variational Quantum Algorithm

In [ ]:
import numpy as np
import quantumcircuitsimulator.circuit

from scipy.optimize import minimize
from quantumcircuitsimulator.circuit import run_program
from quantumcircuitsimulator.experiment import get_counts

In [ ]:
# Set the number of qubits
n = 3

In [ ]:
target_distr = np.random.rand(2**n)
target_distr /= sum(target_distr)

In [ ]:
initial_state = quantumcircuitsimulator.circuit.get_ground_state(n)
print(f'initial state = {initial_state}')

program = [
    { "gate": "h",  "target": [0]    },
    { "gate": "cx", "target": [0, 1] },
    { "gate": "u3", "params": { "theta": "global_1", "phi": "global_2", "lambda_angle": -3.1415 }, "target": [0] },
]

def objective_function(params):
    final_state = run_program(initial_state, program, { "global_1": params[0], "global_2": params[1] })
#    print(f'final state = {final_state}')

    NUM_SHOTS = 1000
    counts = get_counts(final_state, NUM_SHOTS)

    def get_probability_distribution(counts):
        distribution = np.zeros(2**n)
        
        for state_index, count in counts.items():
            distribution[int(state_index, 2)] = count

        return distribution/np.sum(distribution)
    
    output_distr = get_probability_distribution(counts)
    
    # ...calculate cost here...
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2**n)])


    return cost

# initial values
params = np.array([3.1415, 1.5708])
objective_function(params)
# minimize
minimum = minimize(objective_function, params, method="Powell", tol=1e-6)

print(f'Values of (theta, phi) that minimise the state = {minimum.x}')

### References
With thanks to [Qiskit-Textbook](http://community.qiskit.org/textbook) for approach to calculating the cost function.